In [1]:
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.model_selection import train_test_split
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

unable to import 'smart_open.gcs', disabling that module


In [65]:
ted = pd.read_csv('data/ted_expanded.csv')
ted = ted.drop(list(ted.loc[ted['transcript'].isna() == True].index))

In [66]:
ted.loc[:, 'laugh_count'] = ted['transcript'].str.count("(Laughter)")
ted.loc[:, 'applause_count'] = ted['transcript'].str.count("(Applause)")
ted.loc[:, 'days_since_publish'] = ((pd.to_datetime(['2017-09-25']).astype(int) / 10**9) - ted['published_date']) / (60 * 60 * 24)
ted.loc[:, 'lifetime_virality'] = ted['views'] / ted['days_since_publish']
ted.loc[:, 'super_popular'] = 0
ted.loc[ted['views'] > 4000000, 'super_popular'] = 1
ted.loc[ted['speaker_occupation'].str.contains('Artist|artist') == True, 'speaker_occupation'] = "Artist"
ted.loc[ted['speaker_occupation'].str.contains('Author|author|Writer|writer') == True, 'speaker_occupation'] = "Writer"
ted.loc[ted['speaker_occupation'].isin(small_occupations) == True, 'speaker_occupation'] = "Other"
# ted['transcript'] = (ted['transcript'].str.replace("\\'", "")
#                      .str.replace("\(Laughter\)", " ")
#                      .str.replace("\(Applause\)", " ")
#                      .str.lower()
#                      .str.translate(str.maketrans('()—', ' '*len('()—')))
#                      .str.translate(str.maketrans('', '', string.punctuation))
#                      .str.translate(str.maketrans('', '', string.digits)))

In [28]:
# transcripts = (ted['transcript'].apply(word_tokenize)
#                                 .apply(
#                                     lambda x: [item for item in x if item not in stopwords.words('english')]
#                                 ))

In [67]:
sifted = ['description', 'duration', 'event', 'main_speaker', 'name', 'speaker_occupation',
          'title', 'transcript', 'laugh_count', 'applause_count', 'days_since_publish', 'super_popular']
ted = ted[sifted]

In [68]:
train = ted.loc[ted['days_since_publish'] > 250]
test = ted.loc[ted['days_since_publish'] <= 250]

In [69]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop(['super_popular'], axis = 1), train[['super_popular']], 
    test_size = 0.2, random_state = 42)